In [37]:
import joblib
joblib.dump(algorithms["Naive Bayes"], 'stockmarket.joblib')
best_model = joblib.load('stockmarket.joblib')
best_model

GaussianNB()

In [68]:
predict1 = pd.DataFrame([[1939.369995, 0, 1939.369995, 1939.369995]], columns=X_train.columns)
predict1 = scale(predict1, ["Open", "Volume", "Low", "High"], standard_scaler)

predict2 = pd.DataFrame([[16504.99023, 112200000, 16385.0293, 16515.83984]], columns=X_train.columns)
predict2 = scale(predict2, ["Open", "Volume", "Low", "High"], standard_scaler)

predict3 = pd.DataFrame([[16504.99023, 0, 1253.869995, 1253.869995]], columns=X_train.columns)
predict3 = scale(predict3, ["Open", "Volume", "Low", "High"], standard_scaler)

predict4 = pd.DataFrame([[18191.89063, 88700000, 18191.89063, 18295.26953]], columns=X_train.columns)
predict4 = scale(predict4, ["Open", "Volume", "Low", "High"], standard_scaler)

In [69]:
predictions = best_model.predict(pd.concat([predict1, predict2, predict3, predict4]))
for values, prediction in zip(["predict1", "predict2", "predict3", "predict4"], predictions):
    print(f"{values} {prediction}")

predict1 1939
predict2 16465
predict3 22898
predict4 18221
predict5 21457


Die Voraussagen des Algorithmus sind überraschender Weise relativ genau. Es gibt manche Abweichungen, die sind jedoch nicht sehr extrem und für mich in einem akzeptablem Bereich. Es hat darunter auch Vorhersagen welche genau die korrekten Werte hatte. Für die Test Daten habe ich einfach zufällige Daten aus dem Test Datensatz ausgewählt.

In [59]:
X_test.head(15)

,Open,Volume,Low,High
12732,0.451176,1.327331,0.455107,0.442964
1078,-1.356795,-0.623586,-1.356749,-1.356941
7794,0.971610,-0.623586,0.986268,0.983679
6109,2.697485,-0.623586,2.724992,2.756426
8166,0.464563,-0.623586,0.472635,0.488621
360,-1.411423,-0.623586,-1.411747,-1.411239
977,-1.388955,-0.623586,-1.389127,-1.388907
8289,0.283807,-0.623586,0.235389,0.274600
3613,-0.796243,-0.623586,-0.792401,-0.799777
8693,0.936257,-0.623586,0.951835,0.955842
